# Day 7: No Space Left On Device

## Part 1

In [ ]:
from aoc_2023 import core


_example = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""
_test = core.read_input("../data/day_7.txt")

In [ ]:
from typing import Optional
from dataclasses import dataclass


@dataclass(frozen=True)
class Node:
    parent: Optional['Node']
    name: str
    size: int
    children: dict[str, 'Node']

In [ ]:
def update_tree(root: Node, current: Node, line: str) -> Node:
    if line.startswith("$ ls"):
        return current
    elif line.startswith("$ cd"):
        arg = line[5:]
        if arg == "/":
            return root
        elif arg == "..":
            return current.parent
        else:
            child = current.children.get(
                arg,
                Node(
                    parent=current,
                    name=arg,
                    size=0,
                    children=dict()
                ))
            current.children[arg] = child
            return child
    else:
        info, name = line.split(" ")
        if not name in current.children:
            current.children[name] = Node(
                parent=current,
                name=name,
                size=0 if info == "dir" else int(info),
                children=dict())
        return current
                                             

def dfs(node: Node) -> list[Node]:
    return [node] + sum([dfs(child) for child in node.children.values()], [])
    
    
def subtree_size(node: Node) -> int:
    return sum(node.size for node in dfs(node))


def build_tree(s: str) -> Node:
    current = root = Node(
        parent=None,
        name="/",
        size=0,
        children=dict())
    lines = [line for line in s.split("\n") if line]
    for line in lines:
        current = update_tree(root, current, line)
    return root

In [ ]:
def part_1(s: str) -> int:
    root = build_tree(s)
    return sum(
        _size for node in dfs(root) 
        if (node.children) and ((_size := subtree_size(node)) <= 100000)
    )

In [ ]:
part_1(_example)

95437

In [ ]:
part_1(_test)

1444896

## Part 2

In [ ]:
def part_2(s: str) -> int:
    root = build_tree(s)
    disk_size = 70000000
    space_required = 30000000
    min_size = space_required - (disk_size - subtree_size(root))
    
    return sorted([
        _size for node in dfs(root)
        if node.children and ((_size := subtree_size(node)) >= min_size)])[0]

In [ ]:
part_2(_example)

24933642

In [ ]:
part_2(_test)

404395